# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: `Аверьянов Никита Сергеевич`

Группа: `DS-12`

In [2]:
import warnings
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import os
from math import inf
from random import uniform
import requests
warnings.filterwarnings('ignore')

import folium
from shapely import geometry
from shapely.geometry import shape, Polygon, mapping, MultiPolygon, LineString, Point
from shapely.ops import unary_union, cascaded_union, transform
import pyproj
from pyproj import CRS
from haversine import haversine
import rtree.index
import pygeos
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
from OSMPythonTools.nominatim import Nominatim
import osmapi as osm
from openrouteservice import client

#import mapclassify
#from matplotlib import cm
#from matplotlib.colors import ListedColormap
#import zipfile
#import re
#from PIL import Image
#from io import BytesIO

%matplotlib inline

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

#### 0. Task preparation

In [3]:
def get_four_points(lat, long, step, lat_min, lat_max):
    
    return [(min(lat + step / 50, lat_max), long), 
            (max(lat - step / 50, lat_min), long), 
            (lat, long + step if long + step <= 180 else long + step - 360), 
            (lat, long - step if long - step >= -180 else long - step + 360)]

def make_step(afg_centroid, rand_point):
    return haversine(afg_centroid, rand_point) / np.sqrt(2) / 111

def get_nearest_point(four_points, afg_centroid):
    dist = list(map(lambda x: haversine(x, afg_centroid), four_points))
    return np.argmin(dist)

afg_gpd = gpd.read_file('AFG_adm/AFG_adm2.shp')
afg_gpd = afg_gpd.drop([255])

polygons = []
for row in afg_gpd.iterrows():
    temp = []
    for coords in row[1].geometry.exterior.coords:
        temp.append((coords[1], coords[0]))
    polygons.append(Polygon(temp))
    
afg_polygon = unary_union(polygons)
afg_map = list(afg_polygon.exterior.coords)

afg_centroid = afg_polygon.centroid.coords[0]

#### 1. Generating random points

In [4]:
lat_min, lat_max = -90., 90.
long_min, long_max = -180., 180.
way_points = []
cur_min_dist = inf
while True:
    rand_lat = uniform(lat_min, lat_max)
    rand_long = uniform(long_min, long_max)
    way_points.append((rand_lat, rand_long))
    if afg_polygon.contains(Point(rand_lat, rand_long)):
        break
    if cur_min_dist < haversine(afg_centroid, (rand_lat, rand_long)):
        continue
    step = make_step(afg_centroid, (rand_lat, rand_long))
    four_points = get_four_points(rand_lat, rand_long, step, lat_min, lat_max)
    point = get_nearest_point(four_points, afg_centroid)

    if cur_min_dist < haversine(afg_centroid, four_points[point]):
        continue

    if point == 0:
        lat_min += step
    elif point == 1:
        lat_max -= step
    elif point == 2:
        long_min = four_points[2][1]
    else:
        long_max = four_points[3][1]
        
goal_point = way_points[-1]

#### 2. Visualization 

In [5]:
my_map = folium.Map(location=(afg_centroid), zoom_start=3)

folium.Polygon(afg_map).add_to(my_map)

for i, p in enumerate(way_points[:-1]):
    folium.Marker(p, popup=f'Point #{i+1}').add_to(my_map)

folium.Marker(goal_point,
              popup='We have finally found it!',
              icon=folium.Icon(color='red')
             ).add_to(my_map)

folium.PolyLine(way_points, color="green", weight=2.0, opacity=0.5).add_to(my_map)

my_map

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

In [6]:
nominatim = Nominatim()
overpass = Overpass()
with open('countries.txt') as fin:
    countries = fin.readlines()
countries = list(map(lambda x: x.strip(), countries))

#### 1. atm + apartments

In [7]:
appartments_counter = dict()

for c in countries:
    areaId = nominatim.query(c).areaId()
    if areaId != None and areaId % 10 == 0:
        query = overpassQueryBuilder(area=areaId, elementType='node', 
                                     selector='"building"="apartments"', out='count')
        result = overpass.query(query)
        appartments_counter[c] = result.countElements()
        query = overpassQueryBuilder(area=areaId, elementType='node', 
                                     selector='"building"="house"', out='count')
        result = overpass.query(query)
        prev = appartments_counter.get(c, 0) 
        appartments_counter[c] = prev + result.countElements()
        
        query = overpassQueryBuilder(area=areaId, elementType='node', 
                                     selector='"building"="residential"', out='count')  
        result = overpass.query(query)
        prev = appartments_counter.get(c, 0) 
        appartments_counter[c] = prev + result.countElements()

In [8]:
res_max_country = sorted(appartments_counter.items(), key = lambda x: x[1])[-1][0]

res_max_country_areaId = nominatim.query(res_max_country).areaId()
query = overpassQueryBuilder(area=res_max_country_areaId, 
                             elementType='node', 
                             selector='"amenity"="atm"', 
                             out='body')
result = overpass.query(query, timeout = 50);

atms = [(x['lat'], x['lon']) for x in result.toJSON()['elements']]
tree = rtree.index.Rtree()
for i, p in enumerate(atms):
    tree.insert(i, p)
    
atm_points = [atms[x] for x in tree.nearest(goal_point, 5)]
answer_1 = sum([haversine(goal_point, atm_p) for atm_p in atm_points])
print(f"""Сумма расстояний от точки до 5 ближайших банкоматов, находящихся в стране с 
наибольшим количеством объектов жилой недвижимости составляет {round(answer_1, 2)} метров""")

Сумма расстояний от точки до 5 ближайших банкоматов, находящихся в стране с 
наибольшим количеством объектов жилой недвижимости составляет 29628.25 метров


#### 2. school + pharmacy

In [9]:
pharmacy_counter = dict()

for c in countries:
    country_area = nominatim.query(c).areaId()
    if country_area != None and country_area % 10 == 0:
        query_capital = overpassQueryBuilder(area=country_area, elementType='node', selector='"capital"="yes"')
        
        q = overpass.query(query_capital).nodes()
        if q:
            capital_name = q[0].tags()['name:en'];
            capital_area = nominatim.query(c).areaId()
            query = overpassQueryBuilder(area=capital_area, elementType='node', 
                                         selector='"amenity"="pharmacy"', out='body')
            result = overpass.query(query, timeout=50);
            pharmacy_counter[c] = result.countElements()

In [10]:
phar_max_country = sorted(pharmacy_counter.items(), key = lambda x: x[1])[-1][0]

phar_max_country_areaId = nominatim.query(phar_max_country).areaId()
query = overpassQueryBuilder(area=phar_max_country_areaId, 
                             elementType='node', 
                             selector='"building"="school"', 
                             out='body')
result = overpass.query(query, timeout = 50);

schools = [(x['lat'], x['lon']) for x in result.toJSON()['elements']]
tree = rtree.index.Rtree()
for i, p in enumerate(schools):
    tree.insert(i, p)
    
school_points = [schools[x] for x in tree.nearest(goal_point, 5)]
answer_2 = sum([haversine(goal_point, school_p) for school_p in school_points])
print(f"""Сумма расстояний от точки до 5 ближайших школ, находящихся в стране с 
наибольшим количеством аптек в столице составляет {round(answer_2, 2)} метров""")

Сумма расстояний от точки до 5 ближайших школ, находящихся в стране с 
наибольшим количеством аптек в столице составляет 58376.11 метров


#### 3. cinema + bus stop + railway station

In [11]:
bustop_dict = dict()
for c in countries:
    areaId = nominatim.query(c).areaId();
    if areaId != None and areaId % 10 == 0:
        print(c)
        query_towns = overpassQueryBuilder(area=nominatim.query(c).areaId(), elementType='node', 
                                   selector='"place"="town"')
        res = overpass.query(query_towns, timeout=200).nodes();
        towns = []
        if res:
            for node in res:
                if node.tags().get('name', 0):
                    towns.append(node.tags()['name'])
            bustop_dict[c] = []
            for town in towns:
                town_areaId = nominatim.query(f'{c}, {town}').areaId()
                query_bus_stops = overpassQueryBuilder(area=areaId, elementType='node', 
                                             selector='"highway"="bus_stop"')
                try:
                    bus_stops = overpass.query(query_bus_stops, timeout=200);
                    bustop_dict[c].extend([(node.lat(), node.lon()) for node in bus_stops.nodes()])
                except:
                    break
bustop_counter = dict()
for c in bustop_dict.keys():
    areaId = nominatim.query(c).areaId();
    if areaId != None and areaId % 10 == 0:
        u = overpassQueryBuilder(area=areaId, elementType='relation', includeGeometry=True, 
                         selector='"admin_level"="2"')
        r = overpass.query(u, timeout=200);
        coords = list(r.toJSON()['elements'][0]['bounds'].values())
        point_1, point_2 = coords[:2], coords[2:]
        res = [(haversine(x, point_1), haversine(x, point_2)) for x in bustop_dict[c]]
        m = np.array([np.argmin(x) for x in res])
        bustop_counter[c] = len(m[m == 0])

Australia
Exception: [overpass] runtime error: Query timed out in "query" at line 1 after 51 seconds.
Azerbaijan
Bangladesh
Brazil
Exception: [overpass] runtime error: Query timed out in "query" at line 1 after 51 seconds.
Brunei
Cameroon
Comoros
Estonia
Ethiopia
Honduras
Kosovo
Liberia
Malawi
Monaco
Norway
Portugal
Sao Tome & Principe
Swaziland
Syria
Tanzania
Uzbekistan


In [12]:
railway_dict = dict()
for c in countries:
    areaId = nominatim.query(c).areaId()
    if areaId != None and areaId % 10 == 0:
        print(c)
        query_towns = overpassQueryBuilder(area=nominatim.query(c).areaId(), elementType='node', 
                                   selector='"place"="town"')
        res = overpass.query(query_towns, timeout=200).nodes();
        towns = []
        if res:
            for node in res:
                if node.tags().get('name', 0):
                    towns.append(node.tags()['name'])
            railway_dict[c] = []
            for town in towns:
                town_areaId = nominatim.query(f'{c}, {town}').areaId()
                query_railway_stops = overpassQueryBuilder(area=areaId, elementType='node', 
                                             selector='"railway"="station"')
                try:
                    railway_station = overpass.query(query_railway_stops, timeout=200);
                    railway_dict[c].extend([(node.lat(), node.lon()) for node in railway_station.nodes()])
                except:
                    break
                    
railway_counter = dict()
for c in railway_dict.keys():
    areaId = nominatim.query(c).areaId()
    if areaId != None and areaId % 10 == 0:
        u = overpassQueryBuilder(area=areaId, elementType='relation', includeGeometry=True, 
                         selector='"admin_level"="2"')
        r = overpass.query(u, timeout=200)
        coords = list(r.toJSON()['elements'][0]['bounds'].values())
        point_1, point_2 = coords[:2], coords[2:]
        res = [(haversine(x, point_1), haversine(x, point_2)) for x in railway_dict[c]]
        m = np.array([np.argmin(x) for x in res])
        railway_counter[c] = len(m[m == 0])

Australia
Azerbaijan
Bangladesh
Brazil
Brunei
Cameroon
Comoros
Estonia
Ethiopia
Honduras
Kosovo
Liberia
Malawi
Monaco
Norway
Portugal
Sao Tome & Principe
Swaziland
Syria
Tanzania
Uzbekistan


In [13]:
result_dict = dict()
for c in railway_counter:
    result_dict[c] = railway_counter[c] / bustop_counter[c] if bustop_counter[c] else 0 
    
ratio_max_country = sorted(result_dict.items(), key = lambda x: x[1])[-1][0]

In [14]:
ratio_max_country_areaId = nominatim.query(ratio_max_country).areaId()
query = overpassQueryBuilder(area=ratio_max_country_areaId, 
                             elementType='node', 
                             selector='"amenity"="cinema"', 
                             out='body')
result = overpass.query(query, timeout = 50);

cinemas = [(x['lat'], x['lon']) for x in result.toJSON()['elements']]
tree = rtree.index.Rtree()
for i, p in enumerate(cinemas):
    tree.insert(i, p)
    
cinema_points = [cinemas[x] for x in tree.nearest(goal_point, 5)]
answer_3 = sum([haversine(goal_point, cinema_p) for cinema_p in cinema_points])
print(f"""Сумма расстояний от точки до 5 ближайших кинотеатров, наодящихся в 
стране с самым большим отношением числа железнодорожных станций к автобусным 
остановкам в южной части составляет {round(answer_3, 2)} метров""")

Сумма расстояний от точки до 5 ближайших кинотеатров, наодящихся в 
стране с самым большим отношением числа железнодорожных станций к автобусным 
остановкам в южной части составляет 19723.45 метров


## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [15]:
api_key = '5b3ce3597851110001cf6248f3b3cf63c69340e3b01ce901115a2fc3' 
clnt = client.Client(key=api_key)

In [16]:
df = pd.read_csv('/home/mo/Geo-data/HW-2/2019_-_2020_School_Point_Locations.csv')
school_coords = df.the_geom.apply(lambda a: (float(a.split()[1][1:]), float(a.split()[2][:-1])))

In [17]:
boundaries = [-74.02358, -73.92232, 40.69328, 40.81342]
cleaned_school_coords = []
for school in school_coords :
    if school[0] < boundaries[0] or \
    school[0] > boundaries[1] or \
    school[1] < boundaries[2] or \
    school[1] > boundaries[3]:
        pass
    else:
        cleaned_school_coords.append(school)

def make_buffer_for_school(school, radius, resolution=16):    
    wgs = pyproj.Proj(init='epsg:4326')
    utm = pyproj.Proj(init='epsg:32632')
    buffers_proj = Point(pyproj.transform(wgs, utm, *school)).buffer(radius, resolution)
    buffer = []
    for point in buffers_proj.exterior.coords:
        buffer.append(pyproj.transform(utm, wgs, *point))
    return buffer

buffers = []
school_polygons = []
for i, school in enumerate(cleaned_school_coords):
    buffer = make_buffer_for_school(school, radius=120, resolution=32)
    poly = Polygon(buffer)
    buffers.append(buffer)
    school_polygons.append(poly)
all_polygons = mapping(cascaded_union(school_polygons))

In [18]:
def generate_route(avoided_point_list, coordinates, clnt):
    options = {'coordinates': coordinates, 
               'format_out': 'geojson',
               'profile': 'driving-car',
               'preference': 'shortest',
               'instructions': False,
               'options': {'avoid_polygons': geometry.mapping(MultiPolygon(avoided_point_list))}} 
    return clnt.directions(**options)


def thiсken_route(route, thick=0.001): 
    line_tup = [tuple(line) for line in route['features'][0]['geometry']['coordinates']]
    return LineString(line_tup).buffer(thick)

In [19]:
start_point = [-73.9498065, 40.7967588]
end_point = [-73.9895414, 40.7032046]
coordinates = [start_point, end_point]

my_map_2 = folium.Map(location=list(reversed(start_point)), zoom_start=11)
for coord in coordinates:
    folium.map.Marker(list(reversed(coord))).add_to(my_map_2)
    
folium.Marker(list(reversed(start_point)),
              popup='Central Park Entrance',
              icon=folium.Icon(color='green')).add_to(my_map_2)    

folium.Marker(list(reversed(end_point)),
              popup='Water& Washington Streets Crossing Point',
              icon=folium.Icon(color='red')).add_to(my_map_2)    


avoided_point_list = [] 

base_route = generate_route(avoided_point_list, coordinates, clnt) 
folium.features.GeoJson(data=base_route,
                        name='Base Route',
                        style_function=lambda x: dict(color='#ff0000'),
                        overlay=True).add_to(my_map_2)

thick_base_route = thiсken_route(base_route, 0.01) 

for poly in school_polygons:
    if poly.within(thick_base_route):
        avoided_point_list.append(poly)

new_route = generate_route(avoided_point_list, coordinates, clnt)
thick_new_route = thiсken_route(new_route)

folium.features.GeoJson(data=new_route,
                        name='New Route',
                        style_function=lambda x: dict(color='#00ff00'),
                        overlay=True).add_to(my_map_2)
    
folium.features.GeoJson(data=all_polygons,
                        name='Schools area', 
                        style_function=lambda x: dict(color='#ff00ff', fillcolor='#ff00ff')).add_to(my_map_2)
my_map_2.add_child(folium.map.LayerControl())
my_map_2